#Google Colab. Установка Kafka

In [1]:
!wget https://dlcdn.apache.org/kafka/3.2.0/kafka_2.13-3.2.0.tgz

--2022-07-05 12:05:09--  https://dlcdn.apache.org/kafka/3.2.0/kafka_2.13-3.2.0.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 103955943 (99M) [application/x-gzip]
Saving to: ‘kafka_2.13-3.2.0.tgz’

kafka_2.13-3.2.0.tg 100%[===================>]  99.14M   266MB/s    in 0.4s    

2022-07-05 12:05:09 (266 MB/s) - ‘kafka_2.13-3.2.0.tgz’ saved [103955943/103955943]



In [2]:
!tar -xzf kafka_2.13-3.2.0.tgz

In [3]:
!./kafka_2.13-3.2.0/bin/zookeeper-server-start.sh -daemon ./kafka_2.13-3.2.0/config/zookeeper.properties
!./kafka_2.13-3.2.0/bin/kafka-server-start.sh -daemon ./kafka_2.13-3.2.0/config/server.properties
!echo "Waiting for 10 secs until kafka and zookeeper services are up and running"
!sleep 10

Waiting for 10 secs until kafka and zookeeper services are up and running


In [4]:
!ps -ef | grep kafka

root         446       1 17 12:05 ?        00:00:02 java -Xmx512M -Xms512M -server -XX:+UseG1GC -XX:MaxGCPauseMillis=20 -XX:InitiatingHeapOccupancyPercent=35 -XX:+ExplicitGCInvokesConcurrent -XX:MaxInlineLevel=15 -Djava.awt.headless=true -Xlog:gc*:file=/content/kafka_2.13-3.2.0/bin/../logs/zookeeper-gc.log:time,tags:filecount=10,filesize=100M -Dcom.sun.management.jmxremote -Dcom.sun.management.jmxremote.authenticate=false -Dcom.sun.management.jmxremote.ssl=false -Dkafka.logs.dir=/content/kafka_2.13-3.2.0/bin/../logs -Dlog4j.configuration=file:./kafka_2.13-3.2.0/bin/../config/log4j.properties -cp /content/kafka_2.13-3.2.0/bin/../libs/activation-1.1.1.jar:/content/kafka_2.13-3.2.0/bin/../libs/aopalliance-repackaged-2.6.1.jar:/content/kafka_2.13-3.2.0/bin/../libs/argparse4j-0.7.0.jar:/content/kafka_2.13-3.2.0/bin/../libs/audience-annotations-0.5.0.jar:/content/kafka_2.13-3.2.0/bin/../libs/commons-cli-1.4.jar:/content/kafka_2.13-3.2.0/bin/../libs/commons-lang3-3.8.1.jar:/content/kafka_2.13

In [5]:
!./kafka_2.13-3.2.0/bin/kafka-topics.sh --create --bootstrap-server 127.0.0.1:9092 --replication-factor 1 --partitions 1 --topic test

Created topic test.


In [6]:
!./kafka_2.13-3.2.0/bin/kafka-topics.sh --describe --bootstrap-server 127.0.0.1:9092 --topic test

Topic: test	TopicId: J3Hp4Vl5Rda2MhLGORUW_w	PartitionCount: 1	ReplicationFactor: 1	Configs: segment.bytes=1073741824
	Topic: test	Partition: 0	Leader: 0	Replicas: 0	Isr: 0


In [7]:
!pip install kafka-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 246 kB 15.4 MB/s 


In [8]:
import random

In [9]:
def get_random_value():
    current_sale = {}

    list_city = ['city1', 'city2', 'city3', 'city4', 'city5']
    list_manager = ['manager1', 'manager2', 'manager3']
    list_product = ['product1', 'product2', 'product3']

    current_sale['city'] = random.choice(list_city)
    current_sale['manager'] = random.choice(list_manager)
    current_sale['product'] = random.choice(list_product)
    current_sale['amount'] = random.randint(1, 100)

    return current_sale

In [10]:
from time import sleep
from json import loads, dumps
from kafka import KafkaProducer, KafkaConsumer

In [11]:
producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                             value_serializer=lambda x:dumps(x).encode('utf-8'),
                             compression_type='gzip')

In [12]:
test_topic = 'test'
n_records = 15

In [13]:
for _ in range(n_records):
    future = producer.send(topic = test_topic, value = get_random_value())
    record_metadata = future.get(timeout=10)    
    print('--> The message has been sent to a topic: \
            {}, partition: {}, offset: {}' \
            .format(record_metadata.topic,
                record_metadata.partition,
                record_metadata.offset ))   

--> The message has been sent to a topic:             test, partition: 0, offset: 0
--> The message has been sent to a topic:             test, partition: 0, offset: 1
--> The message has been sent to a topic:             test, partition: 0, offset: 2
--> The message has been sent to a topic:             test, partition: 0, offset: 3
--> The message has been sent to a topic:             test, partition: 0, offset: 4
--> The message has been sent to a topic:             test, partition: 0, offset: 5
--> The message has been sent to a topic:             test, partition: 0, offset: 6
--> The message has been sent to a topic:             test, partition: 0, offset: 7
--> The message has been sent to a topic:             test, partition: 0, offset: 8
--> The message has been sent to a topic:             test, partition: 0, offset: 9
--> The message has been sent to a topic:             test, partition: 0, offset: 10
--> The message has been sent to a topic:             test, partition: 0, o

In [14]:
consumer = KafkaConsumer(
     test_topic,
     bootstrap_servers=['localhost:9092'],
     auto_offset_reset='earliest',
     enable_auto_commit=True,
     group_id='my-group',
     value_deserializer=lambda x: loads(x.decode('utf-8')))

In [15]:
for message in consumer:
    message = message.value
    print('{}'.format(message))

{'city': 'city5', 'manager': 'manager3', 'product': 'product1', 'amount': 45}
{'city': 'city1', 'manager': 'manager1', 'product': 'product2', 'amount': 21}
{'city': 'city1', 'manager': 'manager1', 'product': 'product2', 'amount': 93}
{'city': 'city5', 'manager': 'manager3', 'product': 'product2', 'amount': 2}
{'city': 'city2', 'manager': 'manager2', 'product': 'product1', 'amount': 4}
{'city': 'city2', 'manager': 'manager2', 'product': 'product3', 'amount': 69}
{'city': 'city3', 'manager': 'manager2', 'product': 'product3', 'amount': 79}
{'city': 'city4', 'manager': 'manager2', 'product': 'product2', 'amount': 88}
{'city': 'city3', 'manager': 'manager3', 'product': 'product2', 'amount': 60}
{'city': 'city1', 'manager': 'manager1', 'product': 'product2', 'amount': 25}
{'city': 'city5', 'manager': 'manager1', 'product': 'product3', 'amount': 52}
{'city': 'city3', 'manager': 'manager1', 'product': 'product1', 'amount': 41}
{'city': 'city4', 'manager': 'manager3', 'product': 'product2', 'a

KeyboardInterrupt: ignored